Best For :

TYPE 1 DATASET

1. Logistic regression
**( C=0.1 , l1_ratio = 0.4 , penalty='elasticnet' , solver='saga' , tol=0.12 ,max_iter=1000, class_weight='balanced')**

2. SVM
**( C=1.1,coef0=0,degree=8,gamma='scale', kernel='rbf',tol=0.47,class_weight='balanced' )**

3. XGBoost



Best For :

TYPE 2 DATASET

1. Logistic regression
**( C=0.45  , penalty='l2' , solver='saga' , tol=0.125 , max_iter=1000, class_weight='balanced')**

2. SVM
**( C=2.4,coef0=0.06,degree=7,gamma='scale', kernel='rbf',tol=0.38,class_weight='balanced' )**

3. XGBoost



In [1]:
# warning disable
import warnings
warnings.filterwarnings("ignore")


In [2]:
def RandomCV(param_list,n=100):
    import random
    return iter(random.sample(list(param_list),n))

def check_param(param_list):
    if param_list['solver'] in ['newton-cg', 'lbfgs', 'sag']:
        return param_list['penalty'] in ['l2', 'none']
    elif param_list['solver'] in ['liblinear']:
        return param_list['penalty'] in ['l1', 'l2']
    elif param_list['solver'] in ['saga']:
        return param_list['penalty'] in ['l1', 'l2', 'elasticnet','none']

In [3]:
import logging

from dask.distributed import Client
from dask import delayed, compute

client = Client(threads_per_worker=2, n_workers=4,silence_logs=logging.ERROR)
client

# from dask_saturn import SaturnCluster
# from dask.distributed import Client
# from dask import delayed, compute
#
# import os
# os.environ['SATURN_BASE_URL'] = "https://app.community.saturnenterprise.io"
# os.environ['SATURN_TOKEN'] = "server-4bb72059c7f749af8e03e697f9926845"
#
# cluster = SaturnCluster()
# client = Client(cluster)
# client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 11.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34251,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 11.57 GiB
Comm: tcp://127.0.0.1:36899,Total threads: 2
Dashboard: http://127.0.0.1:42537/status,Memory: 2.89 GiB
Nanny: tcp://127.0.0.1:39781,


In [ ]:
client.close()

### We will be experimenting with the following:

Logistic Regression (JP)
Naive Bayes (SP)
Ensemble Methods (Tree methods) (SK)
XGBoost (SA)
SVM (SA)
Multilayer Perceptron *


In [4]:
import numpy as np
import pandas as pd
from helpers import get_train_test, RANDOM_STATE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    accuracy_score,
    precision_score,
    recall_score,
    confusion_matrix
)

In [5]:
df_1 = pd.read_csv('../dataset/processed_dataset/final_data_S1.csv')

# df_2 = pd.read_csv('../dataset/processed_dataset/final_data_S2.csv')

## Working Type 1 Dataprocessed Dataset

In [6]:
y_1 = df_1['class']
X_1 = df_1.drop(['class'], axis=1)
X_train_1, X_test_1, y_train_1, y_test_1 = get_train_test(X_1, y_1)


In [7]:
print("== Training Values ==")
print(y_train_1.value_counts(normalize=True))
print('\n')
print("== Testing Values ==")
print(y_test_1.value_counts(normalize=True))

== Training Values ==
1    0.75
0    0.25
Name: class, dtype: float64


== Testing Values ==
1    0.75
0    0.25
Name: class, dtype: float64


In [20]:
params = {
    'penalty':['l2','elasticnet'],
    'tol':[x/1000 for x in range(50, 150, 5)],
    'C':[0.1],
    'solver':['saga'],
    'l1_ratio':[x/10 for x in range(2, 6, 1)],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(len(list(param_dict)))

@delayed
def run_model(param, X_train, y_train, X_test, y_test):
    if check_param(param):
        model_obj = LogisticRegression(**param, max_iter=1000,
                                       class_weight='balanced', random_state=RANDOM_STATE)
        model_obj.fit(X_train, y_train)
        param.update({
            "accuracy_score":accuracy_score(y_test, model_obj.predict(X_test)),
            "f1_score":f1_score(y_test, model_obj.predict(X_test)),
            "precision_score":precision_score(y_test, model_obj.predict(X_test)),
            "recall_score":recall_score(y_test, model_obj.predict(X_test)),
            "roc_auc_score":roc_auc_score(y_test, model_obj.predict(X_test)),
            "confusion_matrix":str(confusion_matrix(y_test, model_obj.predict(X_test)))
        })
        return pd.DataFrame(param,index=[0])

# create dask parallel function to run xgb
dfs = (run_model(each, X_train_1, y_train_1, X_test_1, y_test_1) for each in param_dict)

160


In [ ]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
# #
# # not_used = ['random_state','class_weight']
# # final.drop(not_used,axis=1,inplace=
final

In [ ]:
final.to_csv('param/final_data_S1_logistic_regression-test2.csv',index=False)

In [ ]:

params = {
    'C':[x/10 for x in range(1,30,2)],
    'kernel':['rbf'],
    'degree':[x for x in range(5,9,1)],
    'gamma':['scale'],
    'coef0':[x/100 for x in range(0,26,2)],
    'tol':[x/100 for x in range(25,50,2)],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(len(list(param_dict)))

@delayed
def run_model(param, X_train, y_train, X_test, y_test):

    model_obj = SVC(**param, class_weight='balanced',
                    random_state=RANDOM_STATE)
    model_obj.fit(X_train, y_train)
    param.update({
        "accuracy_score":accuracy_score(y_test, model_obj.predict(X_test)),
        "f1_score":f1_score(y_test, model_obj.predict(X_test)),
        "precision_score":precision_score(y_test, model_obj.predict(X_test)),
        "recall_score":recall_score(y_test, model_obj.predict(X_test)),
        "roc_auc_score":roc_auc_score(y_test, model_obj.predict(X_test)),
        "confusion_matrix":str(confusion_matrix(y_test, model_obj.predict(X_test)))
    })
    return pd.DataFrame(param,index=[0])

# create dask parallel function to run xgb
dfs = (run_model(each, X_train_1, y_train_1, X_test_1, y_test_1) for each in param_dict)


In [ ]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
#
# not_used = ['random_state','class_weight']
# final.drop(not_used,axis=1,inplace=
final

In [ ]:
final.to_csv('param/final_data_S1_svm-test4.csv',index=False)

In [10]:
from helpers import XGBoostModel

# model_obj = XGBoostModel(
#     **list(param_dict)[100]
# )
# model_obj.fit(X_train_1, y_train_1, X_test_1, y_test_1, verbose=True)

Training XGBoost model...
[0]	Train-auc:0.81549	Train-logloss:0.66916	Test-auc:0.81171	Test-logloss:0.66914
[1]	Train-auc:0.82837	Train-logloss:0.64866	Test-auc:0.78287	Test-logloss:0.65080
[2]	Train-auc:0.83256	Train-logloss:0.63047	Test-auc:0.80897	Test-logloss:0.63285
[3]	Train-auc:0.84387	Train-logloss:0.61398	Test-auc:0.81848	Test-logloss:0.61472
[4]	Train-auc:0.84670	Train-logloss:0.60110	Test-auc:0.82858	Test-logloss:0.60158
[5]	Train-auc:0.85610	Train-logloss:0.58876	Test-auc:0.83924	Test-logloss:0.58909
[6]	Train-auc:0.86648	Train-logloss:0.57807	Test-auc:0.83780	Test-logloss:0.58204
[7]	Train-auc:0.87550	Train-logloss:0.56724	Test-auc:0.84040	Test-logloss:0.57148
[8]	Train-auc:0.88029	Train-logloss:0.55704	Test-auc:0.84155	Test-logloss:0.56288
[9]	Train-auc:0.89006	Train-logloss:0.54622	Test-auc:0.85193	Test-logloss:0.55445
[10]	Train-auc:0.89224	Train-logloss:0.53666	Test-auc:0.86087	Test-logloss:0.54486
[11]	Train-auc:0.89425	Train-logloss:0.52835	Test-auc:0.86577	Test-logl

In [17]:
[x/100 for x in range(10,100,25)]

[0.1, 0.35, 0.6, 0.85]

In [20]:
# n_estimators = 500
# importance_type = 'gain'
# eval_metric = ['auc','logloss']
# early_stopping_rounds = 20
# scale_pos_weight = pos_ratio(y_train_1)
# max_leaves = 0
# objective = "binary:logistic"

tree_method = 'gpu_hist' #gpu
sampling_method = 'gradient_based' #gpu
colsample_bytree = 0.5
colsample_bylevel = 0.5
colsample_bynode = 0.5
max_depth = 4
learning_rate = 0.1
subsample = 0.5
max_delta_step = 0.5
num_parallel_tree = 1

params = {
    'max_depth':[x for x in range(4,12,4)],
    'grow_policy':['depthwise', 'lossguide'],
    'learning_rate':[x/100 for x in range(0,200,50)],
    'booster':['gbtree'],
    'gamma':[x/100 for x in range(10,100,25)], #min_split_loss
    'min_child_weight':[0.42],
    # 'max_delta_step':[x/100 for x in range(10,50,4)],
    'reg_alpha':[x/100 for x in range(0,50,10)],
    'reg_lambda':[x/100 for x in range(20,100,25)],
    'num_parallel_tree':[x for x in range(1,8,4)],
    # 'tree_method':['exact','hist','approx'],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(param_dict.__len__())

@delayed
def run_model(param, X_train, y_train, X_test, y_test):

    model_obj = XGBoostModel(
        **param,
    )
    model_obj.fit(X_train, y_train, X_test, y_test, verbose=False)
    param.update({
        "accuracy_score":accuracy_score(y_test, model_obj.predict(X_test)),
        "f1_score":f1_score(y_test, model_obj.predict(X_test)),
        "precision_score":precision_score(y_test, model_obj.predict(X_test)),
        "recall_score":recall_score(y_test, model_obj.predict(X_test)),
        "roc_auc_score":roc_auc_score(y_test, model_obj.predict(X_test)),
        "confusion_matrix":str(confusion_matrix(y_test, model_obj.predict(X_test))),
        "best_logloss":model_obj.score(),
        "best_iteration":model_obj.model.best_iteration
    })
    return pd.DataFrame(param,index=[0])

# create dask parallel function to run xgb
dfs = (run_model(each, X_train_1, y_train_1, X_test_1, y_test_1) for each in param_dict)


2560


In [ ]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
#
# not_used = ['random_state','class_weight']
# final.drop(not_used,axis=1,inplace=
final

Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGB

In [11]:
client.close()

/home/shailesh/Desktop/fuse-project-1/venv/lib/python3.8/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/home/shailesh/Desktop/fuse-project-1/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/shailesh/Desktop/fuse-project-1/venv/lib/python3.8/site-packages/xgboost/sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/home/shailesh/Desktop/fuse-project-1/venv/lib/python3.8/site-packages/xgboost/sklearn.py:793: UserWarning: `early_sto

In [15]:
final.to_csv('param/final_data_S1_xgboost-test2.csv',index=False)

## Working Type 2 Dataprocessed Dataset

In [8]:
y_2 = df_2['class']
X_2 = df_2.drop(['class'], axis=1)
X_train_2, X_test_2, y_train_2, y_test_2 = get_train_test(X_2, y_2)


In [9]:
print("== Training Values ==")
print(y_train_2.value_counts(normalize=True))
print('\n')
print("== Testing Values ==")
print(y_test_2.value_counts(normalize=True))

== Training Values ==
1    0.75
0    0.25
Name: class, dtype: float64


== Testing Values ==
1    0.75
0    0.25
Name: class, dtype: float64


In [20]:
params = {
    'penalty':['l2','elasticnet'],
    'tol':[x/1000 for x in range(100, 250, 10)],
    'C':[x/100 for x in range(10, 50, 5)],
    'solver':['saga'],
    'l1_ratio':[x/10 for x in range(2, 5, 1)],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(len(list(param_dict)))

@delayed
def run_model(param, X_train, y_train, X_test, y_test):
    if check_param(param):
        model_obj = LogisticRegression(**param, max_iter=1000,
                                       class_weight='balanced', random_state=RANDOM_STATE)
        model_obj.fit(X_train, y_train)
        param.update({
            "accuracy_score":accuracy_score(y_test, model_obj.predict(X_test)),
            "f1_score":f1_score(y_test, model_obj.predict(X_test)),
            "precision_score":precision_score(y_test, model_obj.predict(X_test)),
            "recall_score":recall_score(y_test, model_obj.predict(X_test)),
            "roc_auc_score":roc_auc_score(y_test, model_obj.predict(X_test)),
            "confusion_matrix":str(confusion_matrix(y_test, model_obj.predict(X_test)))
        })
        return pd.DataFrame(param,index=[0])

# create dask parallel function to run xgb
dfs = (run_model(each, X_train_2, y_train_2, X_test_2, y_test_2) for each in param_dict)

720


In [ ]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
#
# not_used = ['random_state','class_weight']
# final.drop(not_used,axis=1,inplace=
final

In [22]:
final.to_csv('param/final_data_S2_logistic_regression-test2.csv',index=False)

In [44]:
params = {
    'C':[x/10 for x in range(10,30,2)],
    'kernel':['rbf'],
    'degree':[1,3,5,7,9],
    'gamma':['scale'],
    'coef0':[x/100 for x in range(0,26,2)],
    'tol':[x/100 for x in range(20,40,2)],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(param_dict.__len__())

@delayed
def run_model(param, X_train, y_train, X_test, y_test):

    model_obj = SVC(**param, class_weight='balanced',
                    random_state=RANDOM_STATE)
    model_obj.fit(X_train, y_train)
    param.update({
        "accuracy_score":accuracy_score(y_test, model_obj.predict(X_test)),
        "f1_score":f1_score(y_test, model_obj.predict(X_test)),
        "precision_score":precision_score(y_test, model_obj.predict(X_test)),
        "recall_score":recall_score(y_test, model_obj.predict(X_test)),
        "roc_auc_score":roc_auc_score(y_test, model_obj.predict(X_test)),
        "confusion_matrix":str(confusion_matrix(y_test, model_obj.predict(X_test)))
    })
    return pd.DataFrame(param,index=[0])

dfs = (run_model(each , X_train_2, y_train_2, X_test_2, y_test_2) for each in param_dict)


6500


In [45]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
#
# not_used = ['random_state','class_weight']
# final.drop(not_used,axis=1,inplace=
final

,C,coef0,degree,gamma,kernel,tol,accuracy_score,f1_score,precision_score,recall_score,roc_auc_score,confusion_matrix
0,1.0,0.00,1,scale,rbf,0.20,0.823529,0.880000,0.897959,0.862745,0.784314,[[24 10]\n [14 88]]
1,1.0,0.00,1,scale,rbf,0.22,0.823529,0.880000,0.897959,0.862745,0.784314,[[24 10]\n [14 88]]
2,1.0,0.00,1,scale,rbf,0.24,0.830882,0.884422,0.907216,0.862745,0.799020,[[25 9]\n [14 88]]
3,1.0,0.00,1,scale,rbf,0.26,0.830882,0.884422,0.907216,0.862745,0.799020,[[25 9]\n [14 88]]
4,1.0,0.00,1,scale,rbf,0.28,0.816176,0.874372,0.896907,0.852941,0.779412,[[24 10]\n [15 87]]
...,...,...,...,...,...,...,...,...,...,...,...,...
6495,2.8,0.24,9,scale,rbf,0.30,0.882353,0.922330,0.913462,0.931373,0.833333,[[25 9]\n [ 7 95]]
6496,2.8,0.24,9,scale,rbf,0.32,0.889706,0.927536,0.914286,0.941176,0.838235,[[25 9]\n [ 6 96]]
6497,2.8,0.24,9,scale,rbf,0.34,0.897059,0.932692,0.915094,0.950980,0.843137,[[25 9]\n [ 5 97]]
6498,2.8,0.24,9,scale,rbf,0.36,0.875000,0.917073,0.912621,0.921569,0.828431,[[25 9]\n [ 8 94]]


In [46]:
final.to_csv('param/final_data_S2_svm-test2.csv',index=False)

In [ ]:
[x/100 for x in range(10,50,4)]

In [ ]:
from collections import Counter
from xgboost import XGBClassifier
def pos_ratio(Y) -> float:
    """
    Compute the ratio of positive class in the dataset.
    """
    counter = Counter(Y)
    sum_wpos = counter[1]
    sum_wneg = counter[0]

    return sum_wneg / sum_wpos

In [ ]:
model_obj = XGBClassifier(
    **list(param_dict)[100],use_label_encoder=False, random_state=RANDOM_STATE, objective = "binary:logistic",
    n_estimators = 500, max_leaves = 0,  tree_method = 'hist',
    colsample_bytree = 0.5,colsample_bylevel = 0.5,colsample_bynode = 0.5,scale_pos_weight = pos_ratio(y_train_1),
    importance_type = 'gain',max_depth = 4,
    learning_rate = 0.1 ,subsample = 0.5,max_delta_step = 0.5 ,num_parallel_tree = 1
)
model_obj.fit(X_train_1, y_train_1, eval_set=[(X_test_1, y_test_1)], eval_metric=eval_metric, early_stopping_rounds=early_stopping_rounds)

In [ ]:
# n_estimators = 500
# importance_type = 'gain'
# eval_metric = ['auc','logloss']
# early_stopping_rounds = 20
# scale_pos_weight = pos_ratio(y_train_1)
# max_leaves = 0
# objective = "binary:logistic"

tree_method = 'gpu_hist' #gpu
sampling_method = 'gradient_based' #gpu
colsample_bytree = 0.5
colsample_bylevel = 0.5
colsample_bynode = 0.5
max_depth = 4
learning_rate = 0.1
subsample = 0.5
max_delta_step = 0.5
num_parallel_tree = 1

params = {
    # 'max_depth':[x for x in range(4,12,4)],
    'grow_policy':['depthwise', 'lossguide'],
    # 'learning_rate':[x/100 for x in range(0,200,50)],
    'booster':['gbtree', 'gblinear','dart'],
    'gamma':[x/100 for x in range(0,200,50)], #min_split_loss
    'min_child_weight':[x/100 for x in range(10,50,2)],
    # 'max_delta_step':[x/100 for x in range(10,50,4)],
    'reg_alpha':[x/100 for x in range(0,200,50)],
    'reg_lambda':[x/100 for x in range(0,200,50)],
    # 'num_parallel_tree':[x for x in range(1,10,4)],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(param_dict.__len__())

@delayed
def run_model(param, X_train, y_train, X_test, y_test):

    model_obj = XGBClassifier(
        **param,use_label_encoder=False, random_state=RANDOM_STATE, objective = "binary:logistic",n_estimators = 500, max_leaves = 0, scale_pos_weight = pos_ratio(y_train_1), importance_type = 'gain',
        tree_method = 'gpu_hist',sampling_method = 'gradient_based', colsample_bytree = 0.5,colsample_bylevel = 0.5,colsample_bynode = 0.5,
        max_depth = 4, learning_rate = 0.1 ,subsample = 0.5,max_delta_step = 0.5 ,num_parallel_tree = 1
    )
    model_obj.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric=['auc','logloss'], early_stopping_rounds=20)
    param.update({
        "accuracy_score":accuracy_score(y_test, model_obj.predict(X_test)),
        "f1_score":f1_score(y_test, model_obj.predict(X_test)),
        "precision_score":precision_score(y_test, model_obj.predict(X_test)),
        "recall_score":recall_score(y_test, model_obj.predict(X_test)),
        "roc_auc_score":roc_auc_score(y_test, model_obj.predict(X_test)),
        "confusion_matrix":str(confusion_matrix(y_test, model_obj.predict(X_test)))
    })
    return pd.DataFrame(param,index=[0])

# create dask parallel function to run xgb
dfs = (run_model(each, X_train_1, y_train_1, X_test_1, y_test_1) for each in param_dict)


In [ ]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
#
# not_used = ['random_state','class_weight']
# final.drop(not_used,axis=1,inplace=
final

In [ ]:
final.to_csv('param/final_data_S1_svm-test4.csv',index=False)

In [12]:
df_1 = normalize(
    pd.read_csv('../dataset/processed_dataset/final_data_S1.csv'),
    'minmax'
)

df_2 = pd.read_csv('../dataset/processed_dataset/final_data_S2.csv')

## Working Type 1 Dataprocessed Dataset

In [13]:
y_1 = df_1['class']
X_1 = df_1.drop(['class'], axis=1)
X_train_1, X_test_1, y_train_1, y_test_1 = get_train_test(X_1, y_1)


In [14]:
print("== Training Values ==")
print(y_train_1.value_counts(normalize=True))
print('\n')
print("== Testing Values ==")
print(y_test_1.value_counts(normalize=True))

== Training Values ==
1.0    0.746689
0.0    0.253311
Name: class, dtype: float64


== Testing Values ==
1.0    0.743421
0.0    0.256579
Name: class, dtype: float64


In [23]:
params = {
    'penalty':['l1', 'l2', 'elasticnet', 'none'],
    'tol':[x/1000 for x in range(1, 100, 5)],
    'C':[x/100 for x in range(1, 251, 20)],
    'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'l1_ratio':[x/10 for x in range(0, 11)],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(len(list(param_dict)))

@delayed
def run_model(param, X_train, y_train, X_test, y_test):
    if check_param(param):
        model_obj = LogisticRegression(**param, max_iter=1000,
                                       class_weight='balanced', random_state=RANDOM_STATE)
        model_obj.fit(X_train, y_train)
        param.update({
            "accuracy_score":accuracy_score(y_test, model_obj.predict(X_test)),
            "f1_score":f1_score(y_test, model_obj.predict(X_test)),
            "precision_score":precision_score(y_test, model_obj.predict(X_test)),
            "recall_score":recall_score(y_test, model_obj.predict(X_test)),
            "roc_auc_score":roc_auc_score(y_test, model_obj.predict(X_test)),
            "confusion_matrix":confusion_matrix(y_test, model_obj.predict(X_test))
        })
        return pd.DataFrame(param,index=[0]).iloc[0:1,:]

# create dask parallel function to run xgb
dfs = (run_model(each, X_train_1, y_train_1, X_test_1, y_test_1) for each in param_dict)

57200


In [16]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
#
not_used = ['n_jobs','random_state','class_weight']
final.drop(not_used,axis=1,inplace=True)

SyntaxError: invalid syntax (93521726.py, line 1)

In [18]:
client.close()